# Prize-linked savings ticket simulation overview

This notebook simulates a prize-linked savings scenario, analyzing the distribution of matches and the odds of winning a prize.

**How to use this notebook:**
- Use the interactive controls below to set the number of users, mean and standard deviation of tickets per user, and the maximum ticket number.
- Click the green "Save & Run Full Simulation" button to run the simulation.
- The simulation will generate a random winning ticket and simulate ticket purchases for all users.
- The results include:
    - A bar chart showing the distribution of ticket matches.
    - A summary table of ticket match counts.
    - Summary statistics for ticket counts.


In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

def pick_ticket(ticket_size, min_ticket=0, max_ticket=99):
    return random.sample(range(min_ticket, max_ticket + 1), ticket_size)

def generate_num(u, sd):
    return max(0, int(random.gauss(u, sd)))

def count_matches(ticket, winning_ticket):
    return len(set(ticket) & set(winning_ticket))

def run_simulation(num_users, ticket_mean, ticket_sd, ticket_size, max_ticket):
    winning_ticket = pick_ticket(ticket_size, 0, max_ticket)
    user_ticket_dict = {
        str(i): [pick_ticket(ticket_size, 0, max_ticket) for _ in range(min(generate_num(ticket_mean, ticket_sd), 50))] for i in range(num_users)
    }
    match_counts = {}
    for tickets in user_ticket_dict.values():
        for ticket in tickets:
            matches = count_matches(ticket, winning_ticket)
            match_counts[matches] = match_counts.get(matches, 0) + 1
    return match_counts

def plot_match_summary(match_counts, ticket_size):
    import matplotlib.ticker as mticker
    matches = list(range(ticket_size+1))
    ticket_counts = [match_counts.get(m, 0) for m in matches]
    fig, ax = plt.subplots(figsize=(8, 5))
    bars = ax.bar(matches, ticket_counts, color=plt.cm.viridis(np.linspace(0.3, 0.9, len(matches))), edgecolor='black', alpha=0.85)
    ax.set_xlabel('Number of Matches per Ticket', fontsize=14)
    ax.set_ylabel('Number of Tickets', fontsize=14)
    ax.set_title('Prize-linked Savings Match Distribution', fontsize=16, fontweight='bold')
    ax.set_xticks(matches)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    # Add annotation and increase y-axis top margin for clarity
    y_max = max(ticket_counts) * 1.15 if ticket_counts else 1
    ax.set_ylim(0, y_max)
    # Format y-axis with comma separators
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}' if x == int(x) else f'{x:,.0f}'))
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax.annotate(f'{int(height):,}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10, color='black')
    plt.tight_layout()
    plt.show()

## Interactive Simulation Controls
Use the sliders and dropdowns below to adjust simulation parameters and see the results update live.

In [ ]:
# Interactive controls for simulation (simplified, no 'matches to find')
from IPython.display import display, clear_output
import ipywidgets as widgets

num_users_widget = widgets.BoundedIntText(value=10000, min=100, max=1000000, step=100, description='Number of Users:', style={'description_width': '220px'}, layout=widgets.Layout(width='350px'))
ticket_mean_widget = widgets.BoundedIntText(value=25, min=1, max=100, step=1, description='Mean Tickets per User:', style={'description_width': '220px'}, layout=widgets.Layout(width='350px'))
ticket_sd_widget = widgets.BoundedFloatText(value=1, min=0, max=25, step=0.1, description='Tickets per User SD:', style={'description_width': '220px'}, layout=widgets.Layout(width='350px'))
max_ticket_widget = widgets.BoundedIntText(value=99, min=10, max=999, step=1, description='Max Ticket Number (0 to N):', style={'description_width': '220px'}, layout=widgets.Layout(width='350px'))

run_button = widgets.Button(description='Save & Run Full Simulation', button_style='success', layout=widgets.Layout(width='350px'))
output = widgets.Output()

def format_number(val):
    if isinstance(val, float):
        if val.is_integer():
            return f'{int(val):,}'
        else:
            return f'{val:,.5f}'.rstrip('0').rstrip('.')
    elif isinstance(val, int):
        return f'{val:,}'
    else:
        return str(val)

def on_run_button_clicked(b):
    with output:
        clear_output()
        num_users = num_users_widget.value
        ticket_mean = ticket_mean_widget.value
        ticket_sd = ticket_sd_widget.value
        max_ticket = max_ticket_widget.value
        ticket_size = 7  # fixed ticket size
        print(f"Running simulation with: {format_number(num_users)} users, mean {format_number(ticket_mean)}, SD {format_number(ticket_sd)}, max ticket {format_number(max_ticket)}")
        match_counts = run_simulation(num_users, ticket_mean, ticket_sd, ticket_size, max_ticket)
        plot_match_summary(match_counts, ticket_size)
        data = {
            'Number of Matches': list(match_counts.keys()),
            'Ticket Count': list(match_counts.values())
        }
        match_df = pd.DataFrame(data).sort_values('Number of Matches').reset_index(drop=True)
        # Format numbers in the table
        match_df['Number of Matches'] = match_df['Number of Matches'].apply(format_number)
        match_df['Ticket Count'] = match_df['Ticket Count'].apply(format_number)
        print('Ticket Match Summary Table:')
        display(match_df)
        print('\nSummary statistics for ticket counts:')
        # Explanation for each statistic
        stat_explanations = {
            'count': 'Number of different match categories (bars in the chart).',
            'mean': 'Average number of tickets per match category.',
            'std': 'Standard deviation (spread) of ticket counts across match categories.',
            'min': 'Smallest ticket count in any match category.',
            '25%': '25th percentile (lower quartile) of ticket counts.',
            '50%': 'Median (middle value) of ticket counts.',
            '75%': '75th percentile (upper quartile) of ticket counts.',
            'max': 'Largest ticket count in any match category.'
        }
        desc = match_df['Ticket Count'].map(lambda x: int(x.replace(',', ''))).astype(int).describe()
        for stat, val in desc.items():
            explanation = stat_explanations.get(stat, '')
            print(f"{stat}: {format_number(val)}" + (f"  # {explanation}" if explanation else ''))

run_button.on_click(on_run_button_clicked)
display(widgets.VBox([num_users_widget, ticket_mean_widget, ticket_sd_widget, max_ticket_widget, run_button, output]))

In [3]:
# use Python 3.13.5 as kernal

In [4]:
def generate_num(u, sd):
    # Generate a random integer, sampled from a normal distribution
    # with mean `u` and standard deviation `sd`
    import random
    return max(0, int(random.gauss(u, sd)))

In [5]:
def count_matches(ticket, winning_ticket):
    # Count how many numbers in the ticket match the winning ticket
    return len(set(ticket) & set(winning_ticket))